In [1]:
import pandas as pd
import gc
import os
from tqdm import tqdm

In [4]:
import pandas as pd
import os
import gc
from tqdm import tqdm

# ▶ Base 경로
base_path = "마이박스 다운로드\\최종EDA_unique제거 전체\\test\\"

# ▶ 병합 대상 파일 목록
file_names = [
    "test_1_회원정보_모델링용.csv",
    "test_2_신용정보_모델링용.csv",
    "test_3_매출정보_unique_학습용_57제거.csv",
    "test_4.청구입금정보_변동없음.csv",
    "test_5_잔액정보_EDA.csv",
    "test_6.채널정보_unique.csv",
    "test_7_마케팅정보_unique.csv",
    "test_8_성과정보_변동없음_결측치처리.csv"
]
file_paths = [os.path.join(base_path, fname) for fname in file_names]

# ▶ 데이터 타입 최적화 함수
def optimize_types(df):
    for col in df.select_dtypes(include='number').columns:
        if 'float' in str(df[col].dtype):
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif 'int' in str(df[col].dtype):
            df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include='object').columns:
        if df[col].nunique() / len(df) < 0.5:
            df[col] = df[col].astype('category')
    return df

# ▶ 저장 경로
step_path = "병합/merged_step_test.csv"
final_path = "병합/전체 병합_test.csv"

# ▶ 병합 폴더 없으면 생성
os.makedirs("병합", exist_ok=True)

# ▶ 재시작 여부 판단
if os.path.exists(step_path):
    print("✅ 이전 병합 파일을 불러와 이어서 진행합니다.")
    merged = pd.read_csv(step_path, encoding='utf-8-sig')
    start_idx = len(merged.columns) - 1  # ID 포함
else:
    print("🆕 새로운 병합을 시작합니다.")
    df = pd.read_csv(file_paths[0], encoding='utf-8-sig')
    df = optimize_types(df)
    merged = df.copy()
    merged.to_csv(step_path, index=False, encoding='utf-8-sig')
    print(f"✅ 첫 파일 저장 완료: {step_path}")
    del df
    gc.collect()
    start_idx = 1  # 첫 파일 이후부터 시작

# ▶ tqdm으로 병합 진행
for i in tqdm(range(start_idx, len(file_paths)), desc="📁 파일 병합 중", unit="file"):
    file = file_paths[i]
    print(f"\n📄 병합 중: {file_names[i]}")
    df = pd.read_csv(file, encoding='utf-8-sig')
    df = optimize_types(df)

    merged = pd.concat([merged, df.drop(columns=['ID'], errors='ignore')], axis=1)
    merged.to_csv(step_path, index=False, encoding='utf-8-sig')
    print(f"✅ 중간 저장 완료: {step_path} (열 {merged.shape[1]})")

    del df
    gc.collect()

# ▶ 중복 컬럼 제거 (ID 등)
merged = merged.loc[:, ~merged.columns.duplicated()]

# ▶ 최종 저장
merged.to_csv(final_path, index=False, encoding='utf-8-sig')
print(f"\n🎉 최종 병합 완료: {final_path} (행 {merged.shape[0]}, 열 {merged.shape[1]})")

🆕 새로운 병합을 시작합니다.
✅ 첫 파일 저장 완료: 병합/merged_step_test.csv


📁 파일 병합 중:   0%|                                                                         | 0/7 [00:00<?, ?file/s]


📄 병합 중: test_2_신용정보_모델링용.csv


📁 파일 병합 중:  14%|█████████▎                                                       | 1/7 [00:13<01:19, 13.18s/file]

✅ 중간 저장 완료: 병합/merged_step_test.csv (열 104)

📄 병합 중: test_3_매출정보_unique_학습용_57제거.csv


...

📁 파일 병합 중:  86%|███████████████████████████████████████████████████████▋         | 6/7 [04:29<00:50, 50.57s/file]

✅ 중간 저장 완료: 병합/merged_step_test.csv (열 663)

📄 병합 중: test_8_성과정보_변동없음_결측치처리.csv


📁 파일 병합 중: 100%|█████████████████████████████████████████████████████████████████| 7/7 [05:40<00:00, 48.57s/file]

✅ 중간 저장 완료: 병합/merged_step_test.csv (열 711)



🎉 최종 병합 완료: 병합/전체 병합_test.csv (행 600000, 열 704)


- 열어서 확인

In [6]:
df2 = pd.read_csv('병합/전체 병합_test.csv')

In [7]:
df2

,기준년월,ID,남녀구분코드,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,소지카드수_이용가능_신용,입회일자_신용,...,변동률_RV일시불평잔,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M
0,201807,TEST_00000,1,1,1,0,1,2,2,20140501,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.209395,0.231043,0.0,1.332770,1.780392
1,201807,TEST_00001,1,1,1,0,1,1,1,20160201,...,0.999998,1.044473,1.991974,0.999998,0.926569,-0.269161,-0.247241,0.0,0.000000,0.000000
2,201807,TEST_00002,1,1,1,1,1,2,2,20180301,...,0.999998,1.053083,0.999998,0.999998,0.999998,-0.120290,0.029270,0.0,4.123738,5.115589
3,201807,TEST_00003,2,1,1,1,1,1,1,20120701,...,0.999998,1.991630,0.999998,0.999998,0.999998,0.035807,-0.013359,0.0,0.093615,0.349994
4,201807,TEST_00004,2,1,0,1,1,1,1,20031201,...,0.999998,1.053743,0.999998,0.999998,0.999998,-0.538740,-0.449378,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,2,0,0,0,0,0,0,20130301,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.0,0.765533,0.993765
599996,201812,TEST_99996,1,1,1,1,1,1,1,20171001,...,0.999998,0.999998,0.999998,0.999998,0.999998,0.143554,0.233616,0.0,8.564683,11.379632
599997,201812,TEST_99997,2,1,1,1,1,1,1,20180601,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.765533,0.993765
599998,201812,TEST_99998,1,1,1,1,1,3,3,20120201,...,0.999998,0.333183,0.999998,0.999998,0.999998,-0.038153,-0.106142,0.0,-0.241530,0.499079


- 열 개수 train과 안 맞음

In [3]:
# ▶ Base 경로
base_path = "마이박스 다운로드\\최종EDA_unique제거 전체\\test\\"

# ▶ 병합 대상 파일 목록
file_names = [
    "test_1_회원정보_모델링용.csv",
    "test_2_신용정보_모델링용.csv",
    "test_3_매출정보_unique_학습용_57제거.csv",
    "test_4.청구입금정보_변동없음.csv",
    "test_5_잔액정보_EDA.csv",
    "test_6.채널정보_unique.csv",
    "test_7_마케팅정보_unique.csv",
    "test_8_성과정보_변동없음_결측치처리.csv"
]

# ▶ 파일별 행/열 개수 확인
for fname in file_names:
    path = base_path + fname
    try:
        df = pd.read_csv(path)
        print(f"✅ {fname}: {df.shape[0]:,} rows × {df.shape[1]:,} columns")
    except Exception as e:
        print(f"❌ {fname}: 로딩 실패\n  └ 오류 메시지: {e}")


✅ test_1_회원정보_모델링용.csv: 600,000 rows × 65 columns
✅ test_2_신용정보_모델링용.csv: 600,000 rows × 40 columns
✅ test_3_매출정보_unique_학습용_57제거.csv: 600,000 rows × 341 columns
✅ test_4.청구입금정보_변동없음.csv: 600,000 rows × 44 columns
✅ test_5_잔액정보_EDA.csv: 600,000 rows × 73 columns
✅ test_6.채널정보_unique.csv: 600,000 rows × 66 columns
✅ test_7_마케팅정보_unique.csv: 600,000 rows × 40 columns
✅ test_8_성과정보_변동없음_결측치처리.csv: 600,000 rows × 49 columns


### 6.채널정보 train과 열 개수 달라서 수정

In [12]:
df6 = pd.read_csv('마이박스 다운로드\\최종EDA_unique제거 전체\\test\\test_6.채널정보_unique.csv')

In [13]:
df6['방문횟수_모바일웹_B0M'].value_counts()

방문횟수_모바일웹_B0M
0    599958
4        39
5         3
Name: count, dtype: int64

In [14]:
df6.drop(columns = '방문횟수_모바일웹_B0M', inplace=True)
df6

,기준년월,ID,인입횟수_ARS_R6M,이용메뉴건수_ARS_R6M,인입일수_ARS_R6M,인입월수_ARS_R6M,인입후경과월_ARS,인입횟수_ARS_B0M,이용메뉴건수_ARS_B0M,인입일수_ARS_B0M,...,IB문의건수_CL_RV_R6M,IB문의건수_CS_R6M,불만제기후경과월_R12M,당사멤버쉽_방문횟수_B0M,당사멤버쉽_방문횟수_R6M,당사멤버쉽_방문월수_R6M,홈페이지_금융건수_R6M,홈페이지_선결제건수_R6M,홈페이지_금융건수_R3M,홈페이지_선결제건수_R3M
0,201807,TEST_00000,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,201807,TEST_00001,1회 이상,1회 이상,1,1,5,0,0,0,...,0,0,12,0,0,0,0,0,0,0
2,201807,TEST_00002,1회 이상,1회 이상,1,1,3,0,0,0,...,0,0,12,0,2,1,0,0,0,0
3,201807,TEST_00003,1회 이상,1회 이상,2,1,3,0,0,0,...,0,0,12,0,0,0,0,0,0,0
4,201807,TEST_00004,1회 이상,1회 이상,1,1,1,0,0,0,...,0,0,12,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,201812,TEST_99995,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599996,201812,TEST_99996,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,12,0,0,0,3,7,3,3
599997,201812,TEST_99997,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
599998,201812,TEST_99998,1회 이상,1회 이상,0,0,0,0,0,0,...,0,0,12,0,0,0,7,22,4,12


In [ ]:
df6 = pd.read_csv('마이박스 다운로드\\최종EDA_unique제거 전체\\test\\test_6.채널정보_unique.csv', index=False, encoding='utf-8-sig')

---

- 오류가 났을 때 돌려보는 용도

In [4]:
# 데이터프레임을 넣고 column별 특성 및 결측값, 고유값들을 확인하는 함수를 작성해본다.
# 필수는 아니지만 전체적인 흐름을 파악하기 쉬워진다.

def resumetable(df, n):
    print(f'데이터셋 크기: {df.shape}')                                # 데이터프레임의 전체 크기(행, 열) 출력

    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])         # 각 피처의 데이터 타입을 가져와 데이터프레임으로 생성
    summary = summary.reset_index()                                    # 인덱스를 초기화하여 컬럼으로 변환
    summary = summary.rename(columns={'index':'피처'})                 # 'index' 컬럼명을 '피처'로 변경

    summary['결측값 개수'] = df.isnull().sum().values                 # 각 피처의 결측값(null) 개수 계산
    summary['고유값 개수'] = df.nunique().values                      # 각 피처의 고유값 개수 계산
    
    for i in range(n):
        summary[f"{i+1}번째 값"] = df.iloc[i].values
    
    return summary                                                     # 요약 테이블 반환

In [14]:
# del df5
del df3
gc.collect()

0